# Gold Price: Seasonality and Calendar Effects

This notebook investigates seasonality and calendar effects in gold returns and identifies calendar-derived features that may improve forecasting.

**Scope**
- Decompose series into trend/seasonal/residual components
- Analyze return behavior by month, day-of-week, and year-month panels
- Run simple statistical tests (Welch t-tests + bootstrap confidence intervals)
- Recommend practical calendar features for downstream forecasting models

In [ ]:
import os
import warnings
from pathlib import Path

os.environ.setdefault("MPLCONFIGDIR", "/tmp/mpl-cache")
os.environ.setdefault("XDG_CACHE_HOME", "/tmp/xdg-cache")

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import ttest_ind, f_oneway, kruskal
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.stats.multitest import multipletests

plt.style.use("seaborn-v0_8-whitegrid")
sns.set_theme(style="whitegrid", context="notebook", palette="deep")

pd.set_option("display.max_columns", 120)
pd.set_option("display.float_format", lambda x: f"{x:,.6f}")

RNG = np.random.default_rng(42)
DATA_PATH = Path("gold_historical_data.csv")

In [ ]:
if not DATA_PATH.exists():
    raise FileNotFoundError(f"Dataset not found: {DATA_PATH.resolve()}")

df = pd.read_csv(DATA_PATH, parse_dates=["Date"]).sort_values("Date").reset_index(drop=True)

df["Return"] = df["Close"].pct_change()
df["LogReturn"] = np.log(df["Close"]).diff()
df["Year"] = df["Date"].dt.year
df["Month"] = df["Date"].dt.month
df["MonthName"] = df["Date"].dt.strftime("%b")
df["Weekday"] = df["Date"].dt.day_name()

analysis_df = df.dropna(subset=["Return"]).copy()

print(f"Rows: {len(df):,}")
print(f"Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
print(f"Daily return observations: {len(analysis_df):,}")

df.head()

## 1) Seasonality and Regime View (Monthly Return Heatmap)

In [ ]:
monthly_close = df.set_index("Date")["Close"].resample("ME").last()
monthly_ret = monthly_close.pct_change()

heat = monthly_ret.to_frame("MonthlyReturn")
heat["Year"] = heat.index.year
heat["Month"] = heat.index.month
heat_pivot = heat.pivot(index="Year", columns="Month", values="MonthlyReturn")

plt.figure(figsize=(14, 6))
sns.heatmap(
    heat_pivot * 100,
    cmap="RdYlGn",
    center=0,
    annot=False,
    cbar_kws={"label": "Monthly Return (%)"},
)
plt.title("Monthly Return Heatmap (Regime View)")
plt.xlabel("Month")
plt.ylabel("Year")
plt.tight_layout()
plt.show()

## 2) Decomposition: Trend + Seasonal + Residual

In [ ]:
# seasonal_decompose needs a regular frequency; use business-day frequency with forward fill.
close_b = df.set_index("Date")["Close"].asfreq("B").ffill()

decomp = seasonal_decompose(close_b, model="additive", period=252)
fig = decomp.plot()
fig.set_size_inches(14, 9)
fig.suptitle("seasonal_decompose on Close (period=252 trading days)", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# STL is often more robust for financial series.
stl_close = STL(close_b, period=252, robust=True).fit()
fig = stl_close.plot()
fig.set_size_inches(14, 9)
fig.suptitle("STL Decomposition on Close (period=252)", y=1.02)
plt.tight_layout()
plt.show()

ret_b = close_b.pct_change().dropna()
stl_ret = STL(ret_b, period=21, robust=True).fit()
fig = stl_ret.plot()
fig.set_size_inches(14, 9)
fig.suptitle("STL Decomposition on Returns (period=21 trading days)", y=1.02)
plt.tight_layout()
plt.show()

## 3) Calendar Effects: Month-of-Year

In [ ]:
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
analysis_df["MonthName"] = pd.Categorical(analysis_df["MonthName"], categories=month_order, ordered=True)

month_stats = (
    analysis_df.groupby(["Month", "MonthName"]) ["Return"]
    .agg(mean="mean", median="median", std="std", count="count")
    .reset_index()
    .sort_values("Month")
)
month_stats["se"] = month_stats["std"] / np.sqrt(month_stats["count"])
month_stats["ci95_low"] = month_stats["mean"] - 1.96 * month_stats["se"]
month_stats["ci95_high"] = month_stats["mean"] + 1.96 * month_stats["se"]
month_stats

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].bar(
    month_stats["MonthName"].astype(str),
    month_stats["mean"] * 100,
    yerr=1.96 * month_stats["se"] * 100,
    color="steelblue",
    alpha=0.9,
    capsize=4,
)
axes[0].axhline(0, color="black", linewidth=1)
axes[0].set_title("Average Daily Return by Month (95% CI)")
axes[0].set_ylabel("Average Return (%)")
axes[0].set_xlabel("Month")

sns.boxplot(
    data=analysis_df,
    x="MonthName",
    y="Return",
    ax=axes[1],
    color="lightgray",
)
axes[1].axhline(0, color="black", linewidth=1)
axes[1].set_title("Return Distribution by Month")
axes[1].set_xlabel("Month")
axes[1].set_ylabel("Daily Return")

plt.tight_layout()
plt.show()

## 4) Calendar Effects: Day-of-Week

In [ ]:
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
analysis_df["Weekday"] = pd.Categorical(analysis_df["Weekday"], categories=weekday_order, ordered=True)

weekday_stats = (
    analysis_df.groupby("Weekday")["Return"]
    .agg(mean="mean", median="median", std="std", count="count")
    .reset_index()
)
weekday_stats["se"] = weekday_stats["std"] / np.sqrt(weekday_stats["count"])
weekday_stats

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(
    weekday_stats["Weekday"].astype(str),
    weekday_stats["mean"] * 100,
    yerr=1.96 * weekday_stats["se"] * 100,
    color="teal",
    alpha=0.9,
    capsize=4,
)
axes[0].axhline(0, color="black", linewidth=1)
axes[0].set_title("Average Daily Return by Day-of-Week (95% CI)")
axes[0].set_ylabel("Average Return (%)")
axes[0].set_xlabel("Weekday")

sns.boxplot(data=analysis_df, x="Weekday", y="Return", ax=axes[1], color="lightgray")
axes[1].axhline(0, color="black", linewidth=1)
axes[1].set_title("Return Distribution by Weekday")
axes[1].set_xlabel("Weekday")
axes[1].set_ylabel("Daily Return")

plt.tight_layout()
plt.show()

## 5) Month + Year Panel (Stability Across Time)

In [ ]:
ym = analysis_df.copy()
ym_group = ym.groupby(["Year", "Month"]) ["Return"].mean().reset_index()
ym_pivot = ym_group.pivot(index="Year", columns="Month", values="Return")

plt.figure(figsize=(14, 6))
sns.heatmap(
    ym_pivot * 100,
    cmap="RdYlGn",
    center=0,
    annot=False,
    cbar_kws={"label": "Mean Daily Return (%)"},
)
plt.title("Year-Month Mean Daily Return Heatmap")
plt.xlabel("Month")
plt.ylabel("Year")
plt.tight_layout()
plt.show()

ym_pivot.tail(5)

## 6) Statistical Tests: Is Calendar Effect Real?

In [ ]:
def bootstrap_mean_diff(x, y, n_boot=3000, rng=None):
    if rng is None:
        rng = np.random.default_rng(42)
    x = np.asarray(x)
    y = np.asarray(y)
    bx = rng.choice(x, size=(n_boot, len(x)), replace=True).mean(axis=1)
    by = rng.choice(y, size=(n_boot, len(y)), replace=True).mean(axis=1)
    diff = bx - by
    ci_low, ci_high = np.quantile(diff, [0.025, 0.975])
    p_boot = 2 * min((diff <= 0).mean(), (diff >= 0).mean())
    return float(diff.mean()), float(ci_low), float(ci_high), float(p_boot)


def effect_table(group_col, value_col="Return", n_boot=3000):
    rows = []
    unique_levels = [lv for lv in analysis_df[group_col].dropna().unique()]

    for level in unique_levels:
        g = analysis_df.loc[analysis_df[group_col] == level, value_col].dropna().values
        r = analysis_df.loc[analysis_df[group_col] != level, value_col].dropna().values

        t_stat, p_val = ttest_ind(g, r, equal_var=False, nan_policy="omit")
        mean_diff = g.mean() - r.mean()
        pooled_std = np.sqrt((g.var(ddof=1) + r.var(ddof=1)) / 2)
        cohen_d = mean_diff / pooled_std if pooled_std > 0 else np.nan

        boot_mean, ci_low, ci_high, p_boot = bootstrap_mean_diff(
            g, r, n_boot=n_boot, rng=np.random.default_rng(42 + abs(hash(str(level))) % 10000)
        )

        rows.append({
            "Level": str(level),
            "N_level": len(g),
            "N_rest": len(r),
            "LevelMean": g.mean(),
            "RestMean": r.mean(),
            "MeanDiff": mean_diff,
            "Cohen_d": cohen_d,
            "t_stat": t_stat,
            "p_value": p_val,
            "boot_diff_mean": boot_mean,
            "boot_ci_low": ci_low,
            "boot_ci_high": ci_high,
            "boot_p_value": p_boot,
        })

    out = pd.DataFrame(rows)
    out["p_adj_fdr_bh"] = multipletests(out["p_value"], method="fdr_bh")[1]
    out["significant_5pct"] = out["p_adj_fdr_bh"] < 0.05
    out["ci_excludes_zero"] = (out["boot_ci_low"] > 0) | (out["boot_ci_high"] < 0)
    out = out.sort_values("p_adj_fdr_bh").reset_index(drop=True)
    return out

month_test = effect_table("MonthName", n_boot=3000)
weekday_test = effect_table("Weekday", n_boot=3000)

month_test

In [ ]:
weekday_test

In [ ]:
# Global omnibus tests for group-level difference.
month_groups = [g["Return"].values for _, g in analysis_df.groupby("MonthName")]
weekday_groups = [g["Return"].values for _, g in analysis_df.groupby("Weekday")]

anova_month = f_oneway(*month_groups)
kruskal_month = kruskal(*month_groups)
anova_weekday = f_oneway(*weekday_groups)
kruskal_weekday = kruskal(*weekday_groups)

global_tests = pd.DataFrame(
    [
        ["Month", "ANOVA", anova_month.statistic, anova_month.pvalue],
        ["Month", "Kruskal", kruskal_month.statistic, kruskal_month.pvalue],
        ["Weekday", "ANOVA", anova_weekday.statistic, anova_weekday.pvalue],
        ["Weekday", "Kruskal", kruskal_weekday.statistic, kruskal_weekday.pvalue],
    ],
    columns=["Group", "Test", "Statistic", "p_value"],
)

global_tests

In [ ]:
# Explicit check for September (or any month of interest) vs all other months.
sep = analysis_df.loc[analysis_df["Month"] == 9, "Return"].dropna().values
non_sep = analysis_df.loc[analysis_df["Month"] != 9, "Return"].dropna().values

sep_t = ttest_ind(sep, non_sep, equal_var=False, nan_policy="omit")
sep_boot = bootstrap_mean_diff(sep, non_sep, n_boot=5000, rng=np.random.default_rng(99))

sep_result = pd.Series(
    {
        "Sep_mean_return": sep.mean(),
        "NonSep_mean_return": non_sep.mean(),
        "MeanDiff_Sep_minus_Others": sep.mean() - non_sep.mean(),
        "Welch_t_stat": sep_t.statistic,
        "Welch_p_value": sep_t.pvalue,
        "Bootstrap_diff_mean": sep_boot[0],
        "Bootstrap_CI_low": sep_boot[1],
        "Bootstrap_CI_high": sep_boot[2],
        "Bootstrap_p_value": sep_boot[3],
    }
)
sep_result.to_frame("value")

## 7) Feature Recommendations for Forecasting

In [ ]:
sig_month = month_test[(month_test["p_adj_fdr_bh"] < 0.10) & (month_test["ci_excludes_zero"])].copy()
sig_weekday = weekday_test[(weekday_test["p_adj_fdr_bh"] < 0.10) & (weekday_test["ci_excludes_zero"])].copy()

recommendations = []

# Always include low-cost calendar encodings in benchmark models.
recommendations.append({
    "Feature": "Month dummies (one-hot)",
    "Use": "YES",
    "Reason": "Captures fixed month-of-year seasonality and is low complexity.",
})
recommendations.append({
    "Feature": "Weekday dummies (one-hot)",
    "Use": "YES",
    "Reason": "Captures day-of-week effects with minimal model overhead.",
})
recommendations.append({
    "Feature": "Cyclical month encoding (sin/cos)",
    "Use": "YES",
    "Reason": "Preserves cyclic distance between months (Dec close to Jan).",
})

if len(sig_month) > 0:
    recommendations.append({
        "Feature": "Specific month indicators",
        "Use": "PRIORITY",
        "Reason": f"Evidence of month-level effect after FDR correction in: {', '.join(sig_month['Level'].tolist())}",
    })
else:
    recommendations.append({
        "Feature": "Specific month indicators",
        "Use": "OPTIONAL",
        "Reason": "No strong month-specific significance after multiple-testing correction.",
    })

if len(sig_weekday) > 0:
    recommendations.append({
        "Feature": "Specific weekday indicators",
        "Use": "PRIORITY",
        "Reason": f"Evidence of weekday effect after FDR correction in: {', '.join(sig_weekday['Level'].tolist())}",
    })
else:
    recommendations.append({
        "Feature": "Specific weekday indicators",
        "Use": "OPTIONAL",
        "Reason": "No strong weekday-specific significance after multiple-testing correction.",
    })

recommendation_df = pd.DataFrame(recommendations)
recommendation_df

In [ ]:
# Example model-ready calendar feature block.
calendar_features = analysis_df[["Date", "Close", "Return", "Month", "Weekday"]].copy()
calendar_features = pd.get_dummies(calendar_features, columns=["Month", "Weekday"], drop_first=True)
calendar_features["month_sin"] = np.sin(2 * np.pi * analysis_df["Month"] / 12)
calendar_features["month_cos"] = np.cos(2 * np.pi * analysis_df["Month"] / 12)

print("Calendar feature table shape:", calendar_features.shape)
calendar_features.head()

## Final Summary

Interpret the calendar effects using both p-values and effect sizes:
- If significance is weak and effect sizes are small, keep calendar variables as regularized/optional features.
- If specific months or weekdays are consistently significant, prioritize those dummies in forecasting models.
- Validate utility with walk-forward backtests to confirm out-of-sample gain.